# 03 — MART Comuni: RD% vs Rifiuti (ISPRA)

## Domanda civica
**Ci sono comuni che migliorano la raccolta differenziata (%) ma aumentano i rifiuti totali?**

## Cosa fa questo notebook
A partire dal dataset **clean** (output del notebook 02), il notebook produce **due output**:

1) **MART “snapshot” 2020–2023** (1 riga = 1 comune)  
   - RD% 2020 e 2023, variazione in punti percentuali  
   - RU totali 2020 e 2023 (t), variazione  
   - RU pro capite 2020 e 2023 (kg/ab), variazione (supporto interpretativo)  
   - classificazione in quadranti + flag *RD↑ & RU↑* (risposta diretta alla domanda civica)

2) **Serie storica 2019–2023** (formato “long”, 1 riga = comune-anno)  
   - utile per una **seconda pagina Power BI** con trend e approfondimenti temporali

## Output
- `data/mart/mart_comuni_delta_2020_2023_dashboard_IT.csv`
- `data/mart/serie_comuni_rd_ru_2019_2023_powerbi_IT.csv`


In [2]:
from pathlib import Path
import pandas as pd
import numpy as np
import unicodedata

# ======================
# PATHS (modificabili)
# ======================
PROJECT_ROOT = Path("..")  # notebook dentro /notebooks
CLEAN_FILE = PROJECT_ROOT / "data" / "clean" / "ispra_catasto_rifiuti" / "ispra_catasto_rifiuti_clean.parquet"

MART_DIR = PROJECT_ROOT / "data" / "mart"
MART_DIR.mkdir(parents=True, exist_ok=True)
OUT_CSV = MART_DIR / "mart_comuni_delta_2020_2023_dashboard_IT.csv"

OUT_SERIES_CSV = MART_DIR / "serie_comuni_rd_ru_2019_2023_powerbi_IT.csv"

# Anni di confronto
Y0, Y1 = 2020, 2023

print("CLEAN_FILE:", CLEAN_FILE.resolve())
print("OUT_CSV   :", OUT_CSV.resolve())
print("OUT_SERIES:", OUT_SERIES_CSV.resolve())


CLEAN_FILE: C:\Users\matt\OneDrive\Desktop\progetto-pilota\data\clean\ispra_catasto_rifiuti\ispra_catasto_rifiuti_clean.parquet
OUT_CSV   : C:\Users\matt\OneDrive\Desktop\progetto-pilota\data\mart\mart_comuni_delta_2020_2023_dashboard_IT.csv
OUT_SERIES: C:\Users\matt\OneDrive\Desktop\progetto-pilota\data\mart\serie_comuni_rd_ru_2019_2023_powerbi_IT.csv


In [3]:
# ======================
# CARICA DATI PULITI
# ======================
df = pd.read_parquet(CLEAN_FILE)

print("Shape:", df.shape)
print("Colonne:", df.columns.tolist())
df.head()


Shape: (39530, 26)
Colonne: ['codice_comune_istat', 'regione', 'provincia', 'comune', 'popolazione', 'dato_riferito_a', 'frazione_umida1_t', 'verde_t', 'carta_e_cartone_t', 'vetro_t', 'legno_t', 'metallo_t', 'plastica_t', 'raee_t', 'tessili_t', 'selettiva_t', 'rifiuti_da_c_e_d_t', 'pulizia_stradale_a_recupero_t', 'ingombranti_misti_a_recupero_t', 'altro_t', 'raccolta_differenziata_tonnellate', 'ingombranti_a_smaltimento_t', 'indifferenziato_t', 'rifiuti_urbani_tonnellate', 'raccolta_differenziata_perc', 'anno']


,codice_comune_istat,regione,provincia,comune,popolazione,dato_riferito_a,frazione_umida1_t,verde_t,carta_e_cartone_t,vetro_t,...,rifiuti_da_c_e_d_t,pulizia_stradale_a_recupero_t,ingombranti_misti_a_recupero_t,altro_t,raccolta_differenziata_tonnellate,ingombranti_a_smaltimento_t,indifferenziato_t,rifiuti_urbani_tonnellate,raccolta_differenziata_perc,anno
0,01001001,Piemonte,Torino,AGLIE',2621.0,Comune,255.275,363.247,92.683,95.948,...,6.643,2.860,34.280,1.995,NaN,NaN,549.186,NaN,64.73,2019
1,01001002,Piemonte,Torino,AIRASCA,3598.0,Comune,174.489,140.355,239.377,100.730,...,10.801,7.760,110.507,NaN,NaN,NaN,734.180,NaN,62.75,2019
2,01001003,Piemonte,Torino,ALA DI STURA,441.0,Comune,3.240,13.450,23.636,26.368,...,0.159,NaN,30.587,0.005,112.624,NaN,172.960,285.584,39.44,2019
3,01001004,Piemonte,Torino,ALBIANO D'IVREA,1644.0,Comune,175.768,23.606,67.644,59.639,...,15.276,2.775,12.448,3.355,503.335,NaN,192.089,695.424,72.38,2019
4,01001006,Piemonte,Torino,ALMESE,6426.0,Comune,465.505,NaN,352.661,268.956,...,64.082,63.484,96.568,5.621,NaN,NaN,704.745,NaN,80.74,2019


In [4]:
# ======================
# NORMALIZZAZIONI MINIME
# ======================

def norm_istat6(x):
    """Normalizza codice ISTAT a 6 cifre (string)."""
    if pd.isna(x):
        return np.nan
    s = str(x).strip().replace(".0", "")
    s = "".join(ch for ch in s if ch.isdigit())
    if len(s) >= 8:
        s = s.zfill(8)[-6:]
    else:
        s = s.zfill(6)
    return s

def clean_comune_name(s):
    """Nome comune leggibile per Power BI (title case + unicode normalize)."""
    if pd.isna(s):
        return s
    s = str(s).strip()
    s = unicodedata.normalize("NFKC", s)
    s = s.replace("'", "’")
    # se capita roba tipo '??' (encoding rotto), la eliminiamo
    s = s.replace("??", "")
    return s.title()

# ---- mapping colonne (supporta naming diversi) ----
# Cerca una colonna ISTAT
istat_col = None
for c in ["istat_comune_6", "codice_comune_istat", "istat_comune", "IstatComune"]:
    if c in df.columns:
        istat_col = c
        break
if istat_col is None:
    raise KeyError("Non trovo una colonna ISTAT (istat_comune_6 / codice_comune_istat / ...).")

# RU totali (t)
if "totale_ru_t" not in df.columns:
    if "rifiuti_urbani_tonnellate" in df.columns:
        df = df.rename(columns={"rifiuti_urbani_tonnellate": "totale_ru_t"})
    elif "Totale RU (t)" in df.columns:
        df = df.rename(columns={"Totale RU (t)": "totale_ru_t"})

# RD% 
if "percentuale_rd" not in df.columns:
    if "raccolta_differenziata_perc" in df.columns:
        df = df.rename(columns={"raccolta_differenziata_perc": "percentuale_rd"})
    elif "Percentuale RD (%)" in df.columns:
        df = df.rename(columns={"Percentuale RD (%)": "percentuale_rd"})

needed = ["anno","comune","provincia","regione","popolazione","totale_ru_t","percentuale_rd"]
missing = [c for c in needed if c not in df.columns]
if missing:
    raise KeyError(f"Colonne mancanti nel CLEAN: {missing}")

df["istat_comune_6"] = df[istat_col].map(norm_istat6)
df["comune"] = df["comune"].map(clean_comune_name)

# RU pro capite (kg/abitante)
df["ru_pro_capite_kg"] = (df["totale_ru_t"] * 1000) / df["popolazione"]

df.head()


,codice_comune_istat,regione,provincia,comune,popolazione,dato_riferito_a,frazione_umida1_t,verde_t,carta_e_cartone_t,vetro_t,...,ingombranti_misti_a_recupero_t,altro_t,raccolta_differenziata_tonnellate,ingombranti_a_smaltimento_t,indifferenziato_t,totale_ru_t,percentuale_rd,anno,istat_comune_6,ru_pro_capite_kg
0,01001001,Piemonte,Torino,Aglie’,2621.0,Comune,255.275,363.247,92.683,95.948,...,34.280,1.995,NaN,NaN,549.186,NaN,64.73,2019,001001,NaN
1,01001002,Piemonte,Torino,Airasca,3598.0,Comune,174.489,140.355,239.377,100.730,...,110.507,NaN,NaN,NaN,734.180,NaN,62.75,2019,001002,NaN
2,01001003,Piemonte,Torino,Ala Di Stura,441.0,Comune,3.240,13.450,23.636,26.368,...,30.587,0.005,112.624,NaN,172.960,285.584,39.44,2019,001003,647.582766
3,01001004,Piemonte,Torino,Albiano D’Ivrea,1644.0,Comune,175.768,23.606,67.644,59.639,...,12.448,3.355,503.335,NaN,192.089,695.424,72.38,2019,001004,423.007299
4,01001006,Piemonte,Torino,Almese,6426.0,Comune,465.505,NaN,352.661,268.956,...,96.568,5.621,NaN,NaN,704.745,NaN,80.74,2019,001006,NaN


## Approfondimento: serie storica 2019–2023 (per Power BI)

Questo output **non sostituisce** il MART 2020–2023: lo affianca.
Serve per visualizzare l’evoluzione annuale di RD% e rifiuti (totali e pro capite),
per capire se i cambiamenti sono **graduali**, **discontinui** o legati a specifici anni.


In [10]:
# ======================
# SERIE STORICA 2019–2023 (LONG) — ESportazione  POWER BI
# ======================
YEARS_SERIES = [2019, 2020, 2021, 2022, 2023]

series = df[df["anno"].isin(YEARS_SERIES)].copy()

keys = ["istat_comune_6", "regione", "provincia", "comune"]

# consolidiamo eventuali duplicati (1 riga per comune-anno)
series = series.groupby(keys + ["anno"], as_index=False).agg({
    "popolazione": "mean",
    "totale_ru_t": "sum",
    "percentuale_rd": "mean",
    "ru_pro_capite_kg": "mean",
})

# rinomina colonne “friendly”
series = series.rename(columns={
    "percentuale_rd": "percentuale_rd",
    "totale_ru_t": "totale_ru_t",
    "ru_pro_capite_kg": "ru_pro_capite_kg",
})

# Rimuove NaN e infiniti (se ci sono) —> Power BI non li gestisce bene
series["rd_disponibile"] = series["percentuale_rd"].notna()
series["ru_disponibile"] = series["totale_ru_t"].notna()
series["percentuale_rd"] = series["percentuale_rd"].fillna(0)
series["totale_ru_t"] = series["totale_ru_t"].fillna(0)
series["ru_pro_capite_kg"] = series["ru_pro_capite_kg"].fillna(0)

# QA rapido serie
print("Serie storica righe:", series.shape[0])
print("Anni presenti:", sorted(series["anno"].unique().tolist()))
print("Duplicati (istat, anno):", int(series.duplicated(subset=["istat_comune_6","anno"]).sum()))
print("NaN% RD:", round(series["percentuale_rd"].isna().mean()*100, 2))
print("NaN% RU:", round(series["totale_ru_t"].isna().mean()*100, 2))

# export CSV ITA-friendly
series.to_csv(OUT_SERIES_CSV, index=False, sep=";", decimal=",", encoding="utf-8")
print("✅ Saved SERIE:", OUT_SERIES_CSV.resolve())

series.head()


Serie storica righe: 39525
Anni presenti: [2019, 2020, 2021, 2022, 2023]
Duplicati (istat, anno): 0
NaN% RD: 0.0
NaN% RU: 0.0
✅ Saved SERIE: C:\Users\matt\OneDrive\Desktop\progetto-pilota\data\mart\serie_comuni_rd_ru_2019_2023_powerbi_IT.csv


,istat_comune_6,regione,provincia,comune,anno,popolazione,totale_ru_t,percentuale_rd,ru_pro_capite_kg,rd_disponibile,ru_disponibile
0,001001,Piemonte,Torino,Aglie’,2019,2621.0,0.0,64.73,0.0,True,True
1,001001,Piemonte,Torino,Aglie’,2020,2548.0,0.0,66.64,0.0,True,True
2,001001,Piemonte,Torino,Aglie’,2021,2549.0,0.0,62.21,0.0,True,True
3,001001,Piemonte,Torino,Aglie’,2022,2558.0,0.0,65.68,0.0,True,True
4,001001,Piemonte,Torino,Aglie’,2023,2603.0,0.0,62.61,0.0,True,True


In [11]:
# ======================
# FILTRO ANNI E AGGREGAZIONE (1 riga per comune-anno)
# ======================
d = df[df["anno"].isin([Y0, Y1])].copy()

keys = ["istat_comune_6","regione","provincia","comune"]

# In caso di duplicati (può capitare per frazioni / righe non-comune), consolidiamo
d = d.groupby(keys + ["anno"], as_index=False).agg({
    "popolazione": "mean",
    "totale_ru_t": "sum",
    "percentuale_rd": "mean",
    "ru_pro_capite_kg": "mean",
})

print("Shape after groupby:", d.shape)
d.head()


Shape after groupby: (15804, 9)


,istat_comune_6,regione,provincia,comune,anno,popolazione,totale_ru_t,percentuale_rd,ru_pro_capite_kg
0,001001,Piemonte,Torino,Aglie’,2020,2548.0,0.000,66.64,NaN
1,001001,Piemonte,Torino,Aglie’,2023,2603.0,0.000,62.61,NaN
2,001002,Piemonte,Torino,Airasca,2020,3569.0,0.000,61.69,NaN
3,001002,Piemonte,Torino,Airasca,2023,3686.0,0.000,72.24,NaN
4,001003,Piemonte,Torino,Ala Di Stura,2020,448.0,369.621,38.50,825.046875


In [12]:
# ======================
# WIDE 2020 vs 2023 + DELTA
# ======================
w = d.pivot_table(
    index=keys,
    columns="anno",
    values=["percentuale_rd","totale_ru_t","ru_pro_capite_kg","popolazione"],
    aggfunc="first"
).reset_index()

# flatten columns
w.columns = [
    f"{a}_{b}" if isinstance(b, (int, np.integer)) else a
    for a, b in w.columns
]

# rename convenience
w = w.rename(columns={
    f"percentuale_rd_{Y0}": f"percentuale_rd_{Y0}",
    f"percentuale_rd_{Y1}": f"percentuale_rd_{Y1}",
    f"totale_ru_t_{Y0}": f"totale_ru_t_{Y0}",
    f"totale_ru_t_{Y1}": f"totale_ru_t_{Y1}",
    f"ru_pro_capite_kg_{Y0}": f"ru_pro_capite_kg_{Y0}",
    f"ru_pro_capite_kg_{Y1}": f"ru_pro_capite_kg_{Y1}",
    f"popolazione_{Y0}": f"popolazione_{Y0}",
    f"popolazione_{Y1}": f"popolazione_{Y1}",
})

# deltas
w["delta_rd_pp"] = w[f"percentuale_rd_{Y1}"] - w[f"percentuale_rd_{Y0}"]
w["delta_ru_totali_t"] = w[f"totale_ru_t_{Y1}"] - w[f"totale_ru_t_{Y0}"]
w["delta_ru_pro_capite"] = w[f"ru_pro_capite_kg_{Y1}"] - w[f"ru_pro_capite_kg_{Y0}"]

w.head()


,istat_comune_6,regione,provincia,comune,percentuale_rd_2020,percentuale_rd_2023,popolazione_2020,popolazione_2023,ru_pro_capite_kg_2020,ru_pro_capite_kg_2023,totale_ru_t_2020,totale_ru_t_2023,delta_rd_pp,delta_ru_totali_t,delta_ru_pro_capite
0,001001,Piemonte,Torino,Aglie’,66.64,62.61,2548.0,2603.0,NaN,NaN,0.000,0.000,-4.03,0.000,NaN
1,001002,Piemonte,Torino,Airasca,61.69,72.24,3569.0,3686.0,NaN,NaN,0.000,0.000,10.55,0.000,NaN
2,001003,Piemonte,Torino,Ala Di Stura,38.50,44.92,448.0,473.0,825.046875,732.786469,369.621,346.608,6.42,-23.013,-92.260406
3,001004,Piemonte,Torino,Albiano D’Ivrea,70.82,80.69,1650.0,1619.0,411.358788,437.741198,678.742,708.703,9.87,29.961,26.382410
4,001006,Piemonte,Torino,Almese,79.49,77.32,6448.0,6323.0,NaN,NaN,0.000,0.000,-2.17,0.000,NaN


In [13]:
# ======================
# CLASSIFICAZIONE (quadranti) + FLAG DOMANDA CIVICA
# ======================

def quadrante(delta_rd_pp, delta_ru_tot):
    if pd.isna(delta_rd_pp) or pd.isna(delta_ru_tot):
        return "Dati mancanti"
    if delta_rd_pp > 0 and delta_ru_tot <= 0:
        return "Virtuosi (RD↑, RU↓)"
    if delta_rd_pp > 0 and delta_ru_tot > 0:
        return "Migliora RD ma aumenta RU (RD↑, RU↑)"
    if delta_rd_pp <= 0 and delta_ru_tot <= 0:
        return "Riduce RU ma non RD (RD↓, RU↓)"
    return "Peggiora entrambi (RD↓, RU↑)"

w["quadrante"] = w.apply(lambda r: quadrante(r["delta_rd_pp"], r["delta_ru_totali_t"]), axis=1)

# Flag che risponde direttamente alla domanda civica:
w["rd_su_rifiuti_su"] = (w["delta_rd_pp"] > 0) & (w["delta_ru_totali_t"] > 0)

# (Opzionale) Virtuoso strutturale: RD↑ e RU pro capite ↓
w["virtuoso_strutturale"] = (w["delta_rd_pp"] > 0) & (w["delta_ru_pro_capite"] < 0)

w[["comune","delta_rd_pp","delta_ru_totali_t","rd_su_rifiuti_su","quadrante"]].head(10)


,comune,delta_rd_pp,delta_ru_totali_t,rd_su_rifiuti_su,quadrante
0,Aglie’,-4.03,0.000,False,"Riduce RU ma non RD (RD↓, RU↓)"
1,Airasca,10.55,0.000,False,"Virtuosi (RD↑, RU↓)"
2,Ala Di Stura,6.42,-23.013,False,"Virtuosi (RD↑, RU↓)"
3,Albiano D’Ivrea,9.87,29.961,True,"Migliora RD ma aumenta RU (RD↑, RU↑)"
4,Almese,-2.17,0.000,False,"Riduce RU ma non RD (RD↓, RU↓)"
5,Alpette,-1.38,1.821,False,"Peggiora entrambi (RD↓, RU↑)"
6,Alpignano,2.15,0.000,False,"Virtuosi (RD↑, RU↓)"
7,Andezeno,4.69,-32.255,False,"Virtuosi (RD↑, RU↓)"
8,Andrate,5.40,3.301,True,"Migliora RD ma aumenta RU (RD↑, RU↑)"
9,Angrogna,-0.58,0.045,False,"Peggiora entrambi (RD↓, RU↑)"


In [14]:
# ======================
# DEDUPLICAZIONE (1 riga per istat_comune_6)
# ======================
# In alcuni casi ISPRA può avere più righe con lo stesso codice ISTAT.
# Per il MART Power BI vogliamo 1 riga = 1 comune.
# Regola: teniamo la riga con popolazione 2023 più alta (più "affidabile" nei merge).

before = len(w)

pop_col = f"popolazione_{Y1}"
if pop_col not in w.columns:
    # fallback: se manca (non dovrebbe), deduplica senza criterio
    w_dedup = w.drop_duplicates(subset=["istat_comune_6"], keep="first").copy()
else:
    w_dedup = (
        w.sort_values(by=["istat_comune_6", pop_col], ascending=[True, False])
         .drop_duplicates(subset=["istat_comune_6"], keep="first")
         .reset_index(drop=True)
    )

after = len(w_dedup)
print("Righe prima:", before)
print("Righe dopo :", after)
print("Duplicati rimossi:", before - after)

# Da qui in avanti usiamo SEMPRE w_final
w_final = w_dedup


Righe prima: 7926
Righe dopo : 7908
Duplicati rimossi: 18


In [15]:
# ======================
# QA MINIMO
# ======================
print("Comuni (righe):", len(w_final))
print("Quota RD↑ & RU↑ (totali):", round(w_final["rd_su_rifiuti_su"].mean()*100, 3), "%")
print("N RD↑ & RU↑:", int(w_final["rd_su_rifiuti_su"].sum()))
print("N virtuosi strutturali (RD↑ & RU_pc↓):", int(w_final["virtuoso_strutturale"].sum()))

# Duplicati chiave
dup = w_final.duplicated(subset=["istat_comune_6"]).sum()
print("Duplicati per istat_comune_6:", int(dup))

# Range RD
print("RD% min/max 2020:", float(w_final[f"percentuale_rd_{Y0}"].min()), float(w_final[f"percentuale_rd_{Y0}"].max()))
print("RD% min/max 2023:", float(w_final[f"percentuale_rd_{Y1}"].min()), float(w_final[f"percentuale_rd_{Y1}"].max()))


Comuni (righe): 7908
Quota RD↑ & RU↑ (totali): 14.036 %
N RD↑ & RU↑: 1110
N virtuosi strutturali (RD↑ & RU_pc↓): 917
Duplicati per istat_comune_6: 0
RD% min/max 2020: 0.18 100.0
RD% min/max 2023: 0.06 99.92


In [16]:
# ======================
# EXPORT MART (Power BI)
# ======================
out_cols = [
    "istat_comune_6","regione","provincia","comune",
    f"percentuale_rd_{Y0}", f"percentuale_rd_{Y1}", "delta_rd_pp",
    f"totale_ru_t_{Y0}", f"totale_ru_t_{Y1}", "delta_ru_totali_t",
    f"ru_pro_capite_kg_{Y0}", f"ru_pro_capite_kg_{Y1}", "delta_ru_pro_capite",
    "rd_su_rifiuti_su","virtuoso_strutturale","quadrante"
]

mart = w_final[out_cols].copy()

# Salvataggio CSV ITA-friendly
mart.to_csv(OUT_CSV, index=False, sep=";", decimal=",", encoding="utf-8")
print("✅ Saved MART:", OUT_CSV.resolve())

mart.head()


✅ Saved MART: C:\Users\matt\OneDrive\Desktop\progetto-pilota\data\mart\mart_comuni_delta_2020_2023_dashboard_IT.csv


,istat_comune_6,regione,provincia,comune,percentuale_rd_2020,percentuale_rd_2023,delta_rd_pp,totale_ru_t_2020,totale_ru_t_2023,delta_ru_totali_t,ru_pro_capite_kg_2020,ru_pro_capite_kg_2023,delta_ru_pro_capite,rd_su_rifiuti_su,virtuoso_strutturale,quadrante
0,001001,Piemonte,Torino,Aglie’,66.64,62.61,-4.03,0.000,0.000,0.000,NaN,NaN,NaN,False,False,"Riduce RU ma non RD (RD↓, RU↓)"
1,001002,Piemonte,Torino,Airasca,61.69,72.24,10.55,0.000,0.000,0.000,NaN,NaN,NaN,False,False,"Virtuosi (RD↑, RU↓)"
2,001003,Piemonte,Torino,Ala Di Stura,38.50,44.92,6.42,369.621,346.608,-23.013,825.046875,732.786469,-92.260406,False,True,"Virtuosi (RD↑, RU↓)"
3,001004,Piemonte,Torino,Albiano D’Ivrea,70.82,80.69,9.87,678.742,708.703,29.961,411.358788,437.741198,26.382410,True,False,"Migliora RD ma aumenta RU (RD↑, RU↑)"
4,001006,Piemonte,Torino,Almese,79.49,77.32,-2.17,0.000,0.000,0.000,NaN,NaN,NaN,False,False,"Riduce RU ma non RD (RD↓, RU↓)"


## Risposta alla domanda civica

**Ci sono comuni che migliorano la raccolta differenziata (%) ma aumentano i rifiuti totali?**

Sì.  

L’analisi dei dati ISPRA a livello comunale mostra che il miglioramento della raccolta
differenziata **non implica automaticamente** una riduzione dei rifiuti prodotti.

---

### Risultati principali (2020–2023)

- **Comuni analizzati:** 7.926  
- **Comuni con RD in aumento e rifiuti totali in aumento:** **1.110**
- **Quota sul totale:** **14,0%** (circa **1 comune su 7**)

Questi comuni mostrano un miglioramento della percentuale di raccolta differenziata,
ma **continuano a produrre più rifiuti complessivi**.  
Si tratta quindi di un miglioramento **solo parziale**, che non incide sulla quantità
totale di rifiuti generati.

### Note metodologiche

- Il confronto 2020–2023 è scelto per una classificazione “inizio–fine” chiara e adatta a dashboard.
- La serie 2019–2023 aggiunge contesto e consente analisi più approfondite senza introdurre rumore nel messaggio civico.

---

### Approfondimento temporale (2019–2023)
Oltre al MART 2020–2023, viene prodotto un dataset in formato “long” (comune–anno)
per analisi di trend e confronti temporali in Power BI. La serie storica non presenta
duplicati sulla chiave (istat, anno) e ha copertura completa dei rifiuti totali; la RD%
presenta una quota limitata di valori mancanti (~2%).

---

### I “virtuosi strutturali”

- **Comuni con RD in aumento e rifiuti pro capite in diminuzione:** **917**

Questi comuni rappresentano il gruppo realmente virtuoso:
riescono a differenziare di più **e** a ridurre i rifiuti prodotti per abitante.

Il confronto tra i due gruppi evidenzia che:
- i comuni “virtuosi strutturali” sono **meno** di quelli che migliorano solo la RD
- l’aumento della RD, da solo, **non è sufficiente** a ridurre i rifiuti

---

### Lettura civica dei risultati

L’analisi suggerisce che:
- le politiche basate esclusivamente sull’aumento della raccolta differenziata
  rischiano di **non incidere sulla produzione di rifiuti**
- la prevenzione, la riduzione dei consumi e il riuso restano elementi centrali
- la raccolta differenziata è uno strumento necessario, ma **non risolutivo**

---

### Come usare gli output in Power BI

**Pagina 1 — Risposta civica (snapshot 2020–2023)**  
- Scatter: `delta_rd_pp` (X) vs `delta_ru_totali_t` (Y)  
- Evidenzia i comuni con flag `rd_su_rifiuti_su = TRUE` (*RD↑ & RU↑*)  
- KPI: quota e numero di comuni *RD↑ & RU↑* + numero di “virtuosi strutturali” (RD↑ & RU pro capite↓)

**Pagina 2 — Approfondimento (serie 2019–2023)**  
- Line chart / trend: `percentuale_rd` e `ru_pro_capite_kg` per anno  
- Filtri: regione, provincia, comune  
- Obiettivo: capire se i cambiamenti sono progressivi o concentrati in specifici anni